In [72]:
import pandas as pd
import numpy as np
from functools import reduce

In [105]:
file_path = ['../../R98.xlsx', '../../R99.xlsx',
             '../../R1401.xlsx', '../../R1400.xlsx',
             '../../U98.xlsx', '../../U99.xlsx',
             '../../U1401.xlsx', '../../U1400.xlsx']

# Map for accessinng dataFrames of each year easily
file_to_dfs_map = {}

for path in file_path: 
    
    file_identifier = path.split('/')[-1].split('.')[0]
    dataframes_list = []

    excel_file = pd.ExcelFile(path)
    sheetNames = excel_file.sheet_names
    

    for sheet in sheetNames:
        df_name = f'df_{sheet}'  # Create a dynamic DataFrame name
        globals()[df_name] = pd.read_excel(path, sheet_name=sheet)
        dataframes_list.append(df_name)
        print(f'{df_name} DataFrame created with shape: {globals()[df_name].shape}')
    
    file_to_dfs_map[file_identifier] = dataframes_list

df_df_R98Data DataFrame created with shape: (18430, 17)
df_df_R98P1 DataFrame created with shape: (64296, 10)
df_df_R98P2 DataFrame created with shape: (18430, 45)
df_df_R98P3S01 DataFrame created with shape: (516797, 6)
df_df_R98P3S02 DataFrame created with shape: (4175, 6)
df_df_R98P3S03 DataFrame created with shape: (22378, 4)
df_df_R98P3S04 DataFrame created with shape: (68767, 5)
df_df_R98P3S05 DataFrame created with shape: (87170, 4)
df_df_R98P3S06 DataFrame created with shape: (30600, 4)
df_df_R98P3S07 DataFrame created with shape: (35450, 4)
df_df_R98P3S08 DataFrame created with shape: (28627, 4)
df_df_R98P3S09 DataFrame created with shape: (10839, 4)
df_df_R98P3S10 DataFrame created with shape: (0, 4)
df_df_R98P3S11 DataFrame created with shape: (11269, 4)
df_df_R98P3S12 DataFrame created with shape: (67753, 4)
df_df_R98P3S13 DataFrame created with shape: (197127, 3)
df_df_R98P3S14 DataFrame created with shape: (4387, 4)
df_df_R98P4S01 DataFrame created with shape: (10820, 18)

In [107]:
# make new column for sheet 7 part 3 of datas which show the main category of transport cost
for file_name in file_to_dfs_map.keys():
    globals()[file_to_dfs_map[file_name][9]]['code_categorical'] = globals()[file_to_dfs_map[file_name][9]]['code'].astype('str').str[0:3]

In [108]:
#adding label rurall or urban to dataFrame
# 1: Rurall
# 2: urban

for file_name in ['R98', 'R99', 'R1400', 'R1401']:
    globals()[file_to_dfs_map[file_name][9]]['R/U'] = 1

for file_name in ['U98', 'U99', 'U1400', 'U1401']:
    globals()[file_to_dfs_map[file_name][9]]['R/U'] = 0

In [115]:
globals()[file_to_dfs_map[file_name][0]]

,Address,Fasl,weight,khanevartype,Takmil,TakmilDescA,TakmilDescB,TakmilDescC,Jaygozin,JaygozinDescA,JaygozinDescB,JaygozinDescC,BlkAbdJaygozin,RadifJaygozin,IsTel,FVam,province,town
0,10001000119,4,1245,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0,24.0,1,1,Markazi,1
1,10001000120,4,1245,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0,23.0,1,1,Markazi,1
2,10001000126,4,1245,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Markazi,1
3,10001000130,4,1245,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Markazi,1
4,10001000132,4,1245,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Markazi,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19613,13006383821,2,2523,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,Alborz,6
19614,13006383825,2,2523,1,2,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2.0,27.0,2,1,Alborz,6
19615,13006383826,2,2523,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,Alborz,6
19616,13006383832,2,2523,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Alborz,6


In [116]:
for file_name in file_to_dfs_map.keys():
    print(file_name)
    dfs= []

    # suming the one-hot code of category of transport cost
    # reason: number of family members use this fuel
    temp_df = globals()[file_to_dfs_map[file_name][9]].copy()
    temp_df = pd.get_dummies(temp_df, columns= ['code_categorical'])
    temp_df = temp_df.groupby(by= 'Address').agg({
    'value': 'sum',
    'R/U': 'first',
    'code_categorical_722': 'sum',
    'code_categorical_723': 'sum',
    'code_categorical_724': 'sum',
    'code_categorical_731': 'sum',
    'code_categorical_732': 'sum',
    'code_categorical_733': 'sum',
    'code_categorical_734': 'sum'
    }).reset_index()
    
    dfs.append(temp_df)  


    # handling p2_ sum of members that use vehicle, motorcycle, bicycle
    temp = globals()[file_to_dfs_map[file_name][2]].groupby(by=['Address']).agg({
    'vehicle': 'sum',
    'motorcycle': 'sum',
    'bicycle': 'sum'
    }).reset_index().copy()
    dfs.append(temp)

    #encoding genders for calculating number of males in each family
    temp_df = globals()[file_to_dfs_map[file_name][1]].copy()
    temp_df['gender_encoded'] = temp_df['gender'].map({
        'Male': 1,
        'Female': 0
    })
    #encode job for calculating the number of persons that are employed in each family
    temp_df = pd.get_dummies(temp_df, columns= ['occupationalst'])
    # seperate the age of the members
    bins = [0, 20, 30, 50, float('inf')]
    labels = ['<20', '20-30', '30-50', '>50']

    temp_df['age_group'] = pd.cut(temp_df['age'], bins= bins, labels= labels)
    temp_df = pd.get_dummies(temp_df, columns= ['age_group'])

    temp = temp_df.groupby(by= ['Address']).agg({
        'gender_encoded': 'sum',
        'occupationalst_Housewife': 'sum',
        'occupationalst_IncomeWOJob': 'sum',
        'occupationalst_Other': 'sum',
        'occupationalst_Student': 'sum',
        'occupationalst_child': 'sum',
        'occupationalst_employed': 'sum',
        'age_group_<20': 'sum',
        'age_group_20-30': 'sum',
        'age_group_30-50': 'sum',
        'age_group_>50': 'sum'
    }).reset_index()

    dfs.append(temp)

    # adding season of family interview(used for dividing test and train data)
    temp_df = globals()[file_to_dfs_map[file_name][0]][['Address', 'Fasl']]
    dfs.append(temp_df)

    merged_df = reduce(lambda left, right: pd.merge(left, right, on= 'Address', how= 'outer'), dfs)

    merged_df.to_excel(f'{file_name} regress.xlsx')

R98
R99
R1401
R1400
U98
U99
U1401
U1400


In [63]:
globals()[file_to_dfs_map['R98'][1]]['gender_encoded'] = globals()[file_to_dfs_map['R98'][1]]['gender'].map({
    'Male': 1,
    'Female': 0
})
globals()[file_to_dfs_map['R98'][1]] = pd.get_dummies(globals()[file_to_dfs_map['R98'][1]], columns= ['occupationalst'])

# seperate age_group columns for summing number of persons in each family with being in this groups
# in group by section

globals()[file_to_dfs_map['R98'][1]] = pd.get_dummies(globals()[file_to_dfs_map['R98'][1]], columns= ['age_group'])


In [67]:
# seperate the age of the members
bins = [0, 20, 30, 50, float('inf')]
labels = ['<20', '20-30', '30-50', '>50']

globals()[file_to_dfs_map['R98'][1]]['age_group'] = pd.cut(globals()[file_to_dfs_map['R98'][1]]['age'], bins= bins, labels= labels)

In [69]:
# seperate age_group columns for summing number of persons in each family with being in this groups
# in group by section

globals()[file_to_dfs_map['R98'][1]] = pd.get_dummies(globals()[file_to_dfs_map['R98'][1]], columns= ['age_group'])

In [70]:
globals()[file_to_dfs_map['R98'][1]]

,Address,member,relation,gender,age,literacy,studying,degree,maritalst,gender_encoded,...,occupationalst_IncomeWOJob,occupationalst_Other,occupationalst_Student,occupationalst_child,occupationalst_employed,occupationalst_unemployed,age_group_<20,age_group_20-30,age_group_30-50,age_group_>50
0,20010394208,1,Head,Male,50,literate,No,Elemantry,Married,1,...,False,False,False,False,True,False,False,False,True,False
1,20010394208,2,Spouse,Female,45,illiterate,No,illiterate,Married,0,...,False,False,False,False,False,False,False,False,True,False
2,20010394208,3,Child,Female,26,literate,No,Elemantry,Married,0,...,False,False,False,False,False,False,False,True,False,False
3,20010394208,4,Child,Male,25,literate,No,Bachelor,Single,1,...,False,False,False,False,True,False,False,True,False,False
4,20010394208,5,Child,Male,14,literate,Yes,Secondary,Single,1,...,False,False,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64291,23001701017,3,Child,Male,21,literate,No,College,Single,1,...,False,False,False,False,True,False,False,True,False,False
64292,23001701020,1,Head,Male,57,literate,No,Secondary,Married,1,...,False,False,False,False,True,False,False,False,False,True
64293,23001701020,2,Spouse,Female,50,literate,No,Secondary,Married,0,...,False,False,False,False,False,False,False,False,True,False
64294,23001701020,3,Child,Male,26,literate,Yes,Bachelor,Single,1,...,False,False,True,False,False,False,False,True,False,False


In [71]:
globals()[file_to_dfs_map['R98'][1]].groupby(by= ['Address']).agg({
    'gender_encoded': 'sum',
    'occupationalst_Housewife': 'sum',
    'occupationalst_IncomeWOJob': 'sum',
    'occupationalst_Other': 'sum',
    'occupationalst_Student': 'sum',
    'occupationalst_child': 'sum',
    'occupationalst_employed': 'sum',
    'age_group_<20': 'sum',
    'age_group_20-30': 'sum',
    'age_group_30-50': 'sum',
    'age_group_>50': 'sum'
}).reset_index()

,Address,gender_encoded,occupationalst_Housewife,occupationalst_IncomeWOJob,occupationalst_Other,occupationalst_Student,occupationalst_child,occupationalst_employed,age_group_<20,age_group_20-30,age_group_30-50,age_group_>50
0,20001383908,3,1,0,0,0,2,1,1,1,1,0
1,20001383911,1,1,1,0,0,0,0,0,0,0,2
2,20001383915,1,1,0,0,0,0,1,0,0,0,2
3,20001383916,1,1,0,0,1,1,1,2,0,2,0
4,20001383920,4,1,0,0,1,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18425,23006709717,4,1,0,1,0,1,1,1,1,2,1
18426,23006709720,1,1,0,0,0,2,1,2,0,2,0
18427,23006709723,1,1,0,0,1,0,1,0,0,2,1
18428,23006709807,0,0,1,0,0,0,0,0,0,0,1
